In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from google.colab import drive

drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np
import random
import math
import os



In [0]:
#랜덤하게 액션을 선택할 비율 (학습을 진행 하면서 점점 작아짐)
epsilon = 1  
#랜덤하게 선택할 비율의 최소값
epsilonMinimumValue = 0.001 
# 선핵할 액션의 종류 left (0) /stay(1) /right (2) 총 3가지 액션임
nbActions = 3 
#학습을 반복할 횟수
epoch = 2001 
#히든 레이어의 사이즈
hiddenSize = 100
#메모리 사이즈 
maxMemory = 500
#한번에 학습할 데이터의 개수 50개씩 반복해서 학습 
batchSize = 50
#게임 가로 세로 사이즈 10x10 
gridSize = 10 
#화면의 총 픽셀수 10x10
nbStates = gridSize * gridSize 
discount = 0.9 


In [0]:
model = Sequential()
model.add(Dense(hiddenSize, input_dim=100, activation='relu',
                        kernel_initializer='he_uniform'))
model.add(Dense(hiddenSize, activation='relu',
                        kernel_initializer='he_uniform'))
model.add(Dense(nbActions, activation="linear",
                        kernel_initializer='he_uniform'))
model.compile(loss='mse', optimizer=Adam(lr=0.001))

In [0]:

#게임을 진행하는 객체
class CatchEnvironment():
  #객체 생성시 실행
  #self. 이 앞에 있는 변수는 속성으로   CatchEnvironment 의 다른 함수에서
  #사용가능하고 다른 함수에서 값을 변경하면 그 값이 유지됨
  def __init__(self, gridSize):
    #화면의 가로 세로 크기를 저장 gridSize=10  
    self.gridSize = gridSize
    #화면의 전체 셀의 수를 저장 100
    self.nbStates = self.gridSize * self.gridSize
    #현재 과일의 위치,바구니 위치가 저장된 배열 
    self.state = np.empty(3, dtype = np.uint8) 

  #현재 게임 정보를 리턴하는 함수
  def observe(self):
    #canvas 과일의 위치와 바구니의 위치 그리고 나머지 위치를 저장하는 배열
    #canvas배열은 10x10 (화면과 같음)
    #과일이 있는 셀, 바가 있는 셀은 1로 채워져 있고 나머지는 0으로 채워져 있음
    #drawState() : 10x10 배열에 과일 위치, 바구니위치에 1, 나머지 0 리턴하는 함수
    canvas = self.drawState()
    #np.reshape(canvas, (-1,self.nbStates)): nbStates는 100 
    #10x10인 canvas를 1줄 100칸인 2차원 배열로 변환
    canvas = np.reshape(canvas, (-1,self.nbStates))
    return canvas

  def drawState(self):
    #10x10이고 0으로 초기화된  배열 생성
    canvas = np.zeros((self.gridSize, self.gridSize))
    #state[0]: 과일 줄(1부터 시작) state[1]:바구니 칸(1부터 시작)의 인덱스에 0저장
    #인덱스는 0부터 시작하기 때문에 1감소
    canvas[self.state[0]-1, self.state[1]-1] = 1  # Draw the fruit.
    # 바구니는 3개셀을 차지함
    #바구니 칸 state[2](1부터 시작하기 때문에 인덱스는 -1)
    #바구니 줄 (바구니는 화면 맨 아래에 있으므로 9: gridSize(10)-1)

    #바구니 줄,바구니칸 하나 왼쪽 칸에 1대입
    canvas[self.gridSize-1, self.state[2] -1 - 1] = 1
    #바구니 줄 칸에 1대입
    canvas[self.gridSize-1, self.state[2] -1] = 1
    #바구니 줄 칸 오른쪽에 1대입
    canvas[self.gridSize-1, self.state[2] -1 + 1] = 1    
    return canvas        

  # Resets the environment. Randomly initialise the fruit position (always at the top to begin with) and bucket.
  def reset(self): 
    #과일의 칸을 설정 1이상 11 미만 (gridSize+1) 정수 값이 리턴
    initialFruitColumn = random.randrange(1, self.gridSize + 1)
    #과일을 담는 바구니의 위치를 설정 2이상 10 미만의 정수값이 리턴
    #바구니는 3칸을 차지 하기 때문에 2부터 시작
    # |바구니 왼쪽   | 바구니칸(initialBasketPosition)  |바구니 오른쪽    |
    initialBucketPosition = random.randrange(2, self.gridSize + 1 - 1)
    #[과일 줄 (초기값 1), 과일 칸 (랜덤), 바구니칸(랜덤)]을 state에 대입
    self.state = np.array([1, initialFruitColumn, initialBucketPosition]) 
    return self.getState()

  #과일줄,과일칸,바구니 칸 리턴하는 함수  
  def getState(self):
    #state : 과일 줄,과일칸,바구니 칸 저장된 배열
    stateInfo = self.state
    #과일 줄
    fruit_row = stateInfo[0]
    #과일 칸
    fruit_col = stateInfo[1]
    #바구니 칸
    basket = stateInfo[2]
    return fruit_row, fruit_col, basket

  # Returns the award that the agent has gained for being in the current environment state.
  def getReward(self):
    #현재 과일 줄,과일칸,바구니 칸을 리턴받아서
    #fruitRow(과일줄), fruitColumn(과일 칸),basket(바구니 칸)에 대입  
    fruitRow, fruitColumn, basket = self.getState()
    #과일 줄 fruitRow 가 gridSize-1==9 이면 (바구니 줄)
    if (fruitRow == self.gridSize - 1):  # If the fruit has reached the bottom.
      #(fruitColumn - basket): 과일칸과 바구니 칸의 차가 1미만이면  
      if (abs(fruitColumn - basket) <= 1): # Check if the basket caught the fruit.
        return 1 #1리턴
      else:
        return -1#-1 리턴
    else:
      return 0 #0리턴

  def isGameOver(self):
    #state[0]: 과일 줄
    #과일 줄이 9이면  
    if (self.state[0] == self.gridSize - 1): 
      return True #게임 종료
    else: 
      return False #게임 계속
  #바구니 위치 업데이트  
  def updateState(self, action):
    if (action == 1):#왼쪽 선택
      position = -1  #좌로 1칸 이동
    elif (action == 2):#stay 선택
      position = 0 #위치 이동 안함
    else:
      position = 1 #오른쪽으로 1칸 이동
    #과일줄,과일칸,바구니 칸 리턴  
    fruitRow, fruitColumn, basket = self.getState()
    #바구니칸에 1 증가, 바구니 칸이 보다 작으면 2
    newBasket = min(max(2, basket + position), self.gridSize - 1) # The min/max prevents the basket from moving out of the grid.
    #과일 줄 1증가
    fruitRow = fruitRow + 1  # The fruit is falling by 1 every action.
    self.state = np.array([fruitRow, fruitColumn, newBasket])

  #Action can be 1 (move left) or 2 (move right)
  def act(self, action):
    #바구니 위치 업데이트  
    self.updateState(action)
    #reward계산
    reward = self.getReward()
    #게임 종료 여부 리턴
    gameOver = self.isGameOver()
    return self.observe(), reward, gameOver, self.getState()   # For purpose of the visual, I also return the state.




In [0]:
# The memory: Handles the internal memory that we add experiences that occur based on agent's actions,
# and creates batches of experiences based on the mini-batch size for training.
class ReplayMemory:
  #객체 생성시 호출 변수들 초기화  
  def __init__(self, gridSize, maxMemory, discount):
    self.maxMemory = maxMemory
    self.gridSize = gridSize
    self.nbStates = self.gridSize * self.gridSize
    self.discount = discount
    canvas = np.zeros((self.gridSize, self.gridSize))
    canvas = np.reshape(canvas, (-1,self.nbStates))
    self.inputState = np.empty((self.maxMemory, 100), dtype = np.float32)
    self.actions = np.zeros(self.maxMemory, dtype = np.uint8)
    self.nextState = np.empty((self.maxMemory, 100), dtype = np.float32)
    self.gameOver = np.empty(self.maxMemory, dtype = np.bool)
    self.rewards = np.empty(self.maxMemory, dtype = np.int8) 
    self.count = 0
    self.current = 0

  # action(선택한 action), reward(과일 받음 1, 못받음0), nextState (과일과 바구니의 새로운 위치)
  #gameOver:게임 종료 여부 를 배열에 추가
  def remember(self, currentState, action, reward, nextState, gameOver):
    self.actions[self.current] = action
    self.rewards[self.current] = reward
    self.inputState[self.current, ...] = currentState
    self.nextState[self.current, ...] = nextState
    self.gameOver[self.current] = gameOver
    self.count = max(self.count, self.current + 1)
    self.current = (self.current + 1) % self.maxMemory

  #전체 데이터에서 batchSize만큼 선택해서
  #target값 [왼쪽 선택시값, 가만히 있을때 값, 오른쪽 선택시 값] :을 계산해서 리턴
  #게임이 끝나면 해당 action값은 0
  #게임이 진행 중이면 다음 action의 값의 최대값*0.9+나의 reward 1

  #리턴값은 입력과 target을 리턴함
  def getBatch(self,  batchSize, nbActions, nbStates):  
    # We check to see if we have enough memory inputs to make an entire batch, if not we create the biggest
    # batch we can (at the beginning of training we will not have enough experience to fill a batch).
    memoryLength = self.count
    chosenBatchSize = min(batchSize, memoryLength)

    inputs = np.zeros((chosenBatchSize, nbStates))
    targets = np.zeros((chosenBatchSize, nbActions))

    # Fill the inputs and targets up.
    for i in range(chosenBatchSize):
      if memoryLength == 1:
        memoryLength = 2
      # Choose a random memory experience to add to the batch.
      randomIndex = random.randrange(1, memoryLength)
      current_inputState = np.reshape(self.inputState[randomIndex], (1, 100))

      #target = sess.run(model, feed_dict={X: current_inputState})
      target = model.predict(current_inputState)
      current_nextState =  np.reshape(self.nextState[randomIndex], (1, 100))
      #current_outputs = sess.run(model, feed_dict={X: current_nextState})      
      current_outputs = model.predict(current_inputState)
      # Gives us Q_sa, the max q for the next state.
      nextStateMaxQ = np.amax(current_outputs)
      if (self.gameOver[randomIndex] == True):
        target[0, [self.actions[randomIndex]-1]] = self.rewards[randomIndex]
      else:
        # reward + discount(gamma) * max_a' Q(s',a')
        # We are setting the Q-value for the action to  r + gamma*max a' Q(s', a'). The rest stay the same
        # to give an error of 0 for those outputs.
        target[0, [self.actions[randomIndex]-1]] = self.rewards[randomIndex] + self.discount * nextStateMaxQ

      # Update the inputs and targets.
      inputs[i] = current_inputState
      targets[i] = target

    return inputs, targets

In [0]:
print("Training new model")

# Define Environment
env = CatchEnvironment(gridSize)

# Define Replay Memory
memory = ReplayMemory(gridSize, maxMemory, discount)

# Add ops to save and restore all the variables.
#saver = tf.train.Saver()

winCount = 0
#with tf.Session() as sess:	 
    #tf.initialize_all_variables().run() 

for i in range(epoch):
    # Initialize the environment.
    err = 0
    env.reset()
    
    isGameOver = False

    # The initial state of the environment.
    currentState = env.observe()
                
    while (isGameOver != True):
        action = -9999	# action initilization
        # Decides if we should choose a random action, or an action from the policy network.
        global epsilon
        if (np.random.rand() <= epsilon):
            action = random.randrange(1, nbActions+1)
        else:					
            # Forward the current state through the network.
            #q = sess.run(output_layer, feed_dict={X: currentState})					
            q = model.predict(currentState)
            # Find the max index (the chosen action).
            index = q.argmax()
            action = index + 1		 

        # Decay the epsilon by multiplying by 0.999, not allowing it to go below a certain threshold.
        if (epsilon > epsilonMinimumValue):
            epsilon = epsilon * 0.999
        
        nextState, reward, gameOver, stateInfo = env.act(action)
                
        if (reward == 1):
            winCount = winCount + 1

        memory.remember(currentState, action, reward, nextState, gameOver)
        
        # Update the current state and if the game is over.
        currentState = nextState
        isGameOver = gameOver
                        
        # We get a batch of training data to train the model.
        #inputs, targets = memory.getBatch(output_layer, batchSize, nbActions, nbStates, sess, X)
        inputs, targets = memory.getBatch(batchSize, nbActions, nbStates)

        # Train the network which returns the error.
        #_, loss = sess.run([optimizer, cost], feed_dict={X: inputs, Y: targets})
        model.train_on_batch(inputs, targets)	
        
    print("Epoch " + i+ ": Win count = " + str(winCount) + " Win ratio = " + str(float(winCount)/float(i+1)*100))
# # Save the variables to disk.
# save_path = saver.save(sess,"/content/gdrive/My Drive/model.ckpt")
# print("Model saved in file: %s" % save_path)
model.save('/content/gdrive/My Drive/model.h5')

Training new model
Epoch 0: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 100.0
Epoch 1: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 50.0
Epoch 2: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 33.33333333333333
Epoch 3: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 25.0
Epoch 4: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 20.0
Epoch 5: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 16.666666666666664
Epoch 6: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 14.285714285714285
Epoch 7: err = 1.5695375041104853e-05: Win count = 1 Win ratio = 12.5
Epoch 8: err = 1.5695375041104853e-05: Win count = 2 Win ratio = 22.22222222222222
Epoch 9: err = 1.5695375041104853e-05: Win count = 2 Win ratio = 20.0
Epoch 10: err = 1.5695375041104853e-05: Win count = 2 Win ratio = 18.181818181818183
Epoch 11: err = 1.5695375041104853e-05: Win count = 2 Win ratio = 16.666666666666664
Epoch 12: err = 1.5695375041104853e-05: Win count = 3 Wi

In [0]:
%matplotlib
%matplotlib inline

#from TrainCatchGame import CatchEnvironment, X, W1, b1, input_layer, W2, b2, hidden_layer, W3, b3, output_layer, Y, cost, optimizer
from IPython import display
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pylab as pl
import time
import tensorflow as tf
import math
import os
from keras.models import load_model


gridSize = 10 # The size of the grid that the agent is going to play the game on.
maxGames = 100
env = CatchEnvironment(gridSize)
winCount = 0
loseCount = 0
numberOfGames = 0

ground = 1
plot = pl.figure(figsize=(12,12))
axis = plot.add_subplot(111, aspect='equal')
axis.set_xlim([-1, 12])
axis.set_ylim([0, 12])

# Add ops to save and restore all the variables.
#saver = tf.train.Saver()

def drawState(fruitRow, fruitColumn, basket):
	global gridSize
	# column is the x axis
	fruitX = fruitColumn 
	# Invert matrix style points to coordinates
	fruitY = (gridSize - fruitRow + 1)
	statusTitle = "Wins: " + str(winCount) + ",Losses: " + str(loseCount) + ",TotalGame: " + str(numberOfGames)
	axis.set_title(statusTitle, fontsize=30)
	for p in [
		patches.Rectangle(
				((ground - 1), (ground)), 11, 10,
				facecolor="#000000"			# Black
		),
		patches.Rectangle(
				(basket - 1, ground), 2, 0.5,
				facecolor="#FF0000"		 # No background
		),
		patches.Rectangle(
				(fruitX - 0.5, fruitY - 0.5), 1, 1,
				facecolor="#FF0000"			 # red 
		),	 
		]:
			axis.add_patch(p)
	display.clear_output(wait=True)
	display.display(pl.gcf())


#with tf.Session() as sess:		
	# Restore variables from disk.

#saver.restore(sess, "/content/gdrive/My Drive/model.ckpt")
model = load_model('/content/gdrive/My Drive/model.h5')
print('saved model is loaded!')

while (numberOfGames < maxGames):
	numberOfGames = numberOfGames + 1
		
	# The initial state of the environment.
	isGameOver = False
	fruitRow, fruitColumn, basket = env.reset()
	currentState = env.observe()
	drawState(fruitRow, fruitColumn, basket)

	while (isGameOver != True):
		# Forward the current state through the network.
		#q = sess.run(output_layer, feed_dict={X: currentState})
		q = model.predict(currentState)
		# Find the max index (the chosen action).
		index = q.argmax()
		action = index + 1
		nextState, reward, gameOver, stateInfo = env.act(action)		
		fruitRow = stateInfo[0]
		fruitColumn = stateInfo[1]
		basket = stateInfo[2]
		
		# Count game results
		if (reward == 1):
			winCount = winCount + 1
		elif (reward == -1):
			loseCount = loseCount + 1

		currentState = nextState
		isGameOver = gameOver
		drawState(fruitRow, fruitColumn, basket)
		time.sleep(0.4)

display.clear_output(wait=True)